---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    
    # Your Code Here
    acs = [nx.average_clustering(P1_Graphs[i]) for i in range(len(P1_Graphs))]
    asp = [nx.average_shortest_path_length(P1_Graphs[i]) for i in range(len(P1_Graphs))]
    degrees = [P1_Graphs[i].degree() for i in range(len(P1_Graphs))]
    degrees_vals = [sorted(set(degrees[i].values())) for i in range(len(P1_Graphs))]
    degrees_dsb = [[list(degrees[j].values()).count(i) for i in degrees_vals[j]] for j in range(len(P1_Graphs))]
    total_ds = [np.sum(degrees_dsb[i]) for i in range(len(degrees_dsb))]
    per10_ds = [np.sum([degrees_dsb[i][k] for k in range(int(0.5 + 0.1*len(degrees_dsb[i])))]) for i in range(len(degrees_dsb))]
    pa = np.array([], dtype = '<i4')
    sw_l = np.array([], dtype = '<i4')
    sw_h = np.array([], dtype = '<i4')
    gis = np.array([''] * 5, dtype = '<U4')
    for k in range(len(total_ds)):
        if per10_ds[k] > total_ds[k] * 0.5:
            pa = np.append(pa, k)
    graph_df = pd.DataFrame(list(zip(acs, asp)), columns = ['Local Clustering', 'Shortest Path Length']).drop(pa, axis = 0)
    for k in list(graph_df.index):
        if graph_df.loc[k, 'Local Clustering'] < 0.2:
            sw_h = np.append(sw_h, k)
        else:
            sw_l = np.append(sw_l, k)
    gis[pa] = 'PA'
    gis[sw_l] = 'SW_L'
    gis[sw_h] = 'SW_H'
    return list(gis)
#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

AttributeError: 'Graph' object has no attribute '_node'

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    
    # Your Code Here
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import Normalizer
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    Gdf = pd.DataFrame.from_dict(nx.get_node_attributes(G, 'Department'), orient = 'index').rename(columns = {0:'Department'})
    Gdf['Clustering'] = pd.DataFrame.from_dict(nx.clustering(G), orient = 'index')
    ecc = {}
    for d in [nx.eccentricity(g) for g in nx.connected_component_subgraphs(G)]:
        ecc.update(d)
    Gdf['Eccentricity'] = pd.DataFrame.from_dict(ecc, orient = 'index')
    Gdf['Degree'] = pd.DataFrame.from_dict(G.degree(), orient = 'index')
    Gdf['Closeness Centrality'] = pd.DataFrame.from_dict(nx.closeness_centrality(G), orient = 'index')
    Gdf['Betweenness Centrality'] = pd.DataFrame.from_dict(nx.betweenness_centrality(G), orient = 'index')
    Gdf['Page Rank'] = pd.DataFrame.from_dict(nx.pagerank(G, alpha=0.9), orient = 'index')
    h, a = nx.hits(G)
    Gdf['Hubs'] = pd.DataFrame.from_dict(h, orient = 'index')
    Gdf['Authorities'] = pd.DataFrame.from_dict(a, orient = 'index')
    Gdf['ManagementSalary'] = pd.DataFrame.from_dict(nx.get_node_attributes(G, 'ManagementSalary'), orient = 'index')
    
    X_pred = Gdf.where(pd.isnull(Gdf['ManagementSalary'])).dropna(how = 'all', axis = 0).dropna(how = 'all', axis = 1)
    Training_df = Gdf.where(pd.isnull(Gdf['ManagementSalary']) == False).dropna(how = 'all', axis = 0).dropna(how = 'all', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(Training_df.iloc[:, :-1],Training_df.iloc[:, -1], random_state = 0)
    
    #transformer = Normalizer()
    #X_train_norm = transformer.fit_transform(X_train)
    #X_test_norm = transformer.fit_transform(X_test)
    #clf = SVC(C = 1, gamma = 0.1, probability = True, random_state = 0).fit(X_train_norm, y_train)
    
    #clf = RandomForestClassifier(n_estimators=100, random_state=0).fit(X_train, y_train)
    clf = RandomForestClassifier(n_estimators=100, random_state=0).fit(Training_df.iloc[:, :-1],Training_df.iloc[:, -1])
    
    ans_ss = pd.Series([p[1] for p in clf.predict_proba(X_pred)], index = X_pred.index)
    
    #return roc_auc_score(y_test, [p[1] for p in clf.predict_proba(X_test_norm)])
    return ans_ss
#salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import Normalizer
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    
    ncn = {}
    jc = {}
    rai = {}
    aai = {}
    pas = {}
    cnshs = {}
    rashs = {}
    for e in list(future_connections.index):
        ncn.update({e:len(list(nx.common_neighbors(G, e[0], e[1])))})
        
    jaccard = nx.jaccard_coefficient(G, list(future_connections.index))
    for u, v, p in jaccard:
        jc.update({(u, v): p})
        
    ra_index = nx.resource_allocation_index(G, list(future_connections.index))
    for u, v, p in ra_index:
        rai.update({(u, v): p})
        
    aa_index = nx.adamic_adar_index(G, list(future_connections.index))
    for u, v, p in aa_index:
        aai.update({(u, v): p})
        
    pa_score = nx.preferential_attachment(G, list(future_connections.index))
    for u, v, p in pa_score:
        pas.update({(u, v): p})
        
    cnsh_score = nx.cn_soundarajan_hopcroft(G, list(future_connections.index), 'Department')
    for u, v, p in cnsh_score:
        cnshs.update({(u, v): p})
        
    rash_index = nx.ra_index_soundarajan_hopcroft(G, list(future_connections.index), 'Department')
    for u, v, p in rash_index:
        rashs.update({(u, v): p})
        
    Gdf = pd.DataFrame.from_dict(ncn, orient = 'index').rename(columns={0:'Number of Common Neighbors'})
    Gdf['Jaccard Coefficient'] = pd.DataFrame.from_dict(jc, orient = 'index')
    Gdf['Resource Allocation Index'] = pd.DataFrame.from_dict(rai, orient = 'index')
    Gdf['Adamic-Adar Index'] = pd.DataFrame.from_dict(aai, orient = 'index')
    Gdf['Preferential Attachment Score'] = pd.DataFrame.from_dict(pas, orient = 'index')
    Gdf['Community Common Neighbors Score'] = pd.DataFrame.from_dict(cnshs, orient = 'index')
    Gdf['Community Resource Allocation Index'] = pd.DataFrame.from_dict(rashs, orient = 'index')
    Gdf = Gdf.merge(future_connections, how = 'right', left_index = True, right_index = True)
    
    X_pred = Gdf.where(pd.isnull(Gdf['Future Connection'])).dropna(how = 'all', axis = 0).dropna(how = 'all', axis = 1).fillna(-1)
    Training_df = Gdf.where(pd.isnull(Gdf['Future Connection']) == False).dropna(how = 'all', axis = 0).dropna(how = 'all', axis = 1).fillna(-1)
    #X_train, X_test, y_train, y_test = train_test_split(Training_df.iloc[:, :-1],Training_df.iloc[:, -1], random_state = 0)
    
    #clf = RandomForestClassifier(n_estimators=100, random_state=0).fit(X_train, y_train)
    clf = RandomForestClassifier(n_estimators=100, random_state=0).fit(Training_df.iloc[:, :-1],Training_df.iloc[:, -1])
    
    ans_ss = pd.Series([p[1] for p in clf.predict_proba(X_pred)], index = X_pred.index)
    
    return ans_ss#roc_auc_score(y_test, [p[1] for p in clf.predict_proba(X_test)])
new_connections_predictions()

AttributeError: 'Graph' object has no attribute '_node'